In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

patient = pd.read_csv('../../CombinedData/combined1.csv')
allergies = pd.read_csv('../../CombinedData/allergies.csv')
immunizations = pd.read_csv('../../CombinedData/immunizations.csv')
conditions = pd.read_csv('../../CombinedData/conditions.csv')
medications = pd.read_csv('../../CombinedData/medications.csv')
encounters = pd.read_csv('../../CombinedData/encounters.csv')
patient.head()

,ID,BIRTHDATE,DEATHDATE,PASSPORT,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ZIP,...,ALLERGIES_CODE,IMMUNIZATIONS_CODE,CAREPLANS_CODE,CAREPLANS_REASONCODE,BODY_WEIGHT,BODY_HEIGHT,BMI,SYSTOLIC_BP,DIASTOLIC_BP,OBSERVATIONS_CODE
0,660bec03-9e58-47f2-98b9-2f1c564f3838,1996-07-26,NaN,false,NaN,white,irish,F,Fitchburg MA US,20810.0,...,NaN,"[140, 140, 140, 114, 140, 140, 140, 140]","[872781000000100, 266694003, 183051005, 539500...","[39848009.0, 39848009.0, 39848009.0, 10509002....",73.54,163.07,27.66,101.0,75.0,"['8302-2', '29463-7', '39156-5', '8480-6', '84..."
1,5125d2b2-3aef-4ae2-aa5c-335f7e206b92,1996-09-24,NaN,false,NaN,white,french_canadian,F,Westborough MA US,2638.0,...,"[91930004, 419263009, 424213003]","[140, 62, 140, 140, 114, 140, 140, 140, 140]","[326051000000105, 409002, 58332002, 3260510000...","[nan, nan, nan, nan, nan, nan, nan]",17.33,95.92,18.84,137.0,89.0,"['8302-2', '29463-7', '39156-5', '8480-6', '84..."
2,26626faf-cbd5-48d5-a3bf-a7b21ae08e4b,1944-09-01,2015-09-04,X19963891X,M,white,irish,M,Fall River MA US,23401.0,...,NaN,"[133, 140, 33, 140, 140, 140, 140]","[133901003, 385949008, 440381005, 439830001, 3...","[403190006.0, 403190006.0, 403190006.0, 403190...",74.58,174.14,24.60,111.0,71.0,"['8302-2', '29463-7', '39156-5', '8480-6', '84..."
3,eed62b4a-1099-47ec-a2ac-d953830b44d6,1965-08-05,NaN,false,M,hispanic,central_american,M,Shrewsbury MA US,2631.0,...,NaN,"[140, 140, 140, 113]",NaN,NaN,113.23,164.74,41.72,131.0,86.0,"['8302-2', '29463-7', '39156-5', '8480-6', '84..."
4,6e9f8b3e-5a21-401e-868d-2d62e0e7f452,1979-11-03,NaN,X29737332X,M,white,irish,F,Wilmington MA US,2630.0,...,[300913006],"[140, 113, 140]","[326051000000105, 409002, 58332002, 3260510000...","[nan, nan, nan, nan, nan, 72892002.0, 72892002...",106.81,166.13,38.70,131.0,74.0,"['8302-2', '29463-7', '39156-5', '8480-6', '84..."


In [2]:
patient.drop(columns=['PASSPORT','MARITAL','GENDER','BIRTHPLACE','ZIP','BODY_WEIGHT','BODY_HEIGHT','BMI','CONDITIONS_CODE','MEDICATIONS_CODE','MEDICATIONS_REASONCODE','ENCOUNTERS_CODE','ENCOUNTERS_REASONCODE','ALLERGIES_CODE', 'IMMUNIZATIONS_CODE', 'CAREPLANS_CODE', 'CAREPLANS_REASONCODE', 'OBSERVATIONS_CODE'], inplace=True)
conditions.drop(columns=['START', 'STOP', 'ENCOUNTER', 'DESCRIPTION'], inplace=True)
medications.drop(columns=['START', 'STOP', 'ENCOUNTER', 'DESCRIPTION', 'REASONDESCRIPTION', 'REASONCODE'], inplace=True)
immunizations.drop(columns=['DATE', 'ENCOUNTER', 'DESCRIPTION'], inplace=True)
allergies.drop(columns=['START', 'STOP', 'ENCOUNTER', 'DESCRIPTION'], inplace=True)
encounters.drop(columns=['DATE', 'ID', 'DESCRIPTION', 'REASONDESCRIPTION', 'REASONCODE'], inplace=True)

In [3]:
patient.isna().sum()

ID                  0
BIRTHDATE           0
DEATHDATE       97023
RACE               37
ETHNICITY          19
SYSTOLIC_BP         0
DIASTOLIC_BP        0
dtype: int64

In [4]:
# Convert 'BIRTHDATE' to datetime, handling errors by setting problematic values to NaN
patient['BIRTHDATE'] = pd.to_datetime(patient['BIRTHDATE'], errors='coerce')

# Drop rows with NaN values in 'BIRTHDATE'
patient = patient.dropna(subset=['BIRTHDATE'])

# Assuming you have cleaned the 'BIRTHDATE' column, you can proceed with the reference date
reference_date = pd.to_datetime('2023-01-01')

# Calculate age
patient['AGE'] = (reference_date - patient['BIRTHDATE']).dt.days // 365.25
patient.drop(columns=['BIRTHDATE', 'DEATHDATE'], inplace=True)

In [5]:
columns_to_fill_with_mode = ['RACE', 'ETHNICITY']
columns_to_normalize = ['RACE', 'ETHNICITY']
for column in columns_to_fill_with_mode:
    mode_value = patient[column].mode()[0]
    patient[column].fillna(mode_value, inplace=True)
# Create a LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to each column
patient[columns_to_normalize] = patient[columns_to_normalize].apply(label_encoder.fit_transform)

In [6]:
patient.isna().sum()

ID              0
RACE            0
ETHNICITY       0
SYSTOLIC_BP     0
DIASTOLIC_BP    0
AGE             0
dtype: int64

In [7]:
patient.head()

,ID,RACE,ETHNICITY,SYSTOLIC_BP,DIASTOLIC_BP,AGE
0,660bec03-9e58-47f2-98b9-2f1c564f3838,12,29,101.0,75.0,26.0
1,5125d2b2-3aef-4ae2-aa5c-335f7e206b92,12,26,137.0,89.0,26.0
2,26626faf-cbd5-48d5-a3bf-a7b21ae08e4b,12,29,111.0,71.0,78.0
3,eed62b4a-1099-47ec-a2ac-d953830b44d6,10,20,131.0,86.0,57.0
4,6e9f8b3e-5a21-401e-868d-2d62e0e7f452,12,29,131.0,74.0,43.0


In [8]:
midpoint = len(patient) // 2

# Split the DataFrame into two halves based on row index
patient_half1 = patient.iloc[:midpoint]
patient_half2 = patient.iloc[midpoint:]

In [9]:
conditions = conditions.rename(columns={'CODE': 'CONDITIONS_CODE'})
conditions.isna().sum()

PATIENT            0
CONDITIONS_CODE    0
dtype: int64

In [10]:
conditions_code = set([162573006, 254637007, 432121009, 424132000, 422968005, 254632001, 67821000119109])
conditions['TARGET'] = conditions['CONDITIONS_CODE'].apply(lambda code: 1 if code in conditions_code else 0)
conditions.drop(columns=['CONDITIONS_CODE'], inplace=True)
conditions.head()

,PATIENT,TARGET
0,36d131ee-dd5b-4acb-acbe-19961c32c099,0
1,33f33990-ae8b-4be8-938f-e47ad473abfe,0
2,33f33990-ae8b-4be8-938f-e47ad473abfe,0
3,36d131ee-dd5b-4acb-acbe-19961c32c099,0
4,33f33990-ae8b-4be8-938f-e47ad473abfe,0


In [11]:
value_counts = conditions['TARGET'].value_counts()

# Display the result
print("Number of occurrences of each unique value:")
print(value_counts)

Number of occurrences of each unique value:
TARGET
0    474383
1      9079
Name: count, dtype: int64


In [12]:
allergies = allergies.rename(columns={'CODE': 'ALLERGIES_CODE'})
allergies.isna().sum()

PATIENT           0
ALLERGIES_CODE    0
dtype: int64

In [13]:
medications = medications.rename(columns={'CODE': 'MEDICATIONS_CODE'})
medications.isna().sum()

PATIENT             0
MEDICATIONS_CODE    0
dtype: int64

In [14]:
immunizations = immunizations.rename(columns={'CODE': 'IMMUNIZATIONS_CODE'})
immunizations.isna().sum()

PATIENT               0
IMMUNIZATIONS_CODE    0
dtype: int64

In [15]:
encounters = encounters.rename(columns={'CODE': 'ENCOUNTERS_CODE'})
encounters.isna().sum()

PATIENT            0
ENCOUNTERS_CODE    0
dtype: int64

In [16]:
patient_med_df = pd.merge(patient_half1, medications, left_on='ID', right_on='PATIENT', how='inner')
patient_med_df.drop(columns=['PATIENT'], inplace=True)

In [17]:
patient_med_all_df = pd.merge(patient_med_df, allergies, left_on='ID', right_on='PATIENT', how='inner')
patient_med_all_df.drop(columns=['PATIENT'], inplace=True)

In [18]:
patient_med_all_df.shape

(121880, 8)

In [19]:
merged_df = pd.merge(patient_med_all_df, immunizations, left_on='ID', right_on='PATIENT', how='inner')
merged_df.drop(columns=['PATIENT'], inplace=True)

In [20]:
merged_df = pd.merge(merged_df, encounters, left_on='ID', right_on='PATIENT', how='inner')
merged_df.drop(columns=['PATIENT'], inplace=True)

In [21]:
merged_df.shape

(35834651, 10)

In [22]:
import dask.dataframe as dd

# Convert pandas DataFrames to Dask DataFrames
merged_dd = dd.from_pandas(merged_df, npartitions=10)  # Adjust the number of partitions based on your available memory

conditions_dd = dd.from_pandas(conditions, npartitions=10)

In [23]:
# Perform the merge using Dask
merged_dd = dd.merge(merged_dd, conditions_dd, left_on='ID', right_on='PATIENT', how='inner')

In [24]:
# Convert back to pandas DataFrame
merged_df = merged_dd.compute()

In [25]:
merged_df.shape

(207555295, 12)

In [26]:
merged_df.head()

,ID,RACE,ETHNICITY,SYSTOLIC_BP,DIASTOLIC_BP,AGE,MEDICATIONS_CODE,ALLERGIES_CODE,IMMUNIZATIONS_CODE,ENCOUNTERS_CODE,PATIENT,TARGET
0,ea9d0341-21b8-4af5-a19a-f1c4fa5a7f4c,12,36,118.0,86.0,36.0,141918,91934008,140,185349003,ea9d0341-21b8-4af5-a19a-f1c4fa5a7f4c,0
1,ea9d0341-21b8-4af5-a19a-f1c4fa5a7f4c,12,36,118.0,86.0,36.0,141918,91934008,140,185349003,ea9d0341-21b8-4af5-a19a-f1c4fa5a7f4c,0
2,ea9d0341-21b8-4af5-a19a-f1c4fa5a7f4c,12,36,118.0,86.0,36.0,141918,91934008,140,185349003,ea9d0341-21b8-4af5-a19a-f1c4fa5a7f4c,0
3,ea9d0341-21b8-4af5-a19a-f1c4fa5a7f4c,12,36,118.0,86.0,36.0,141918,91934008,140,185349003,ea9d0341-21b8-4af5-a19a-f1c4fa5a7f4c,0
4,ea9d0341-21b8-4af5-a19a-f1c4fa5a7f4c,12,36,118.0,86.0,36.0,141918,91934008,140,308335008,ea9d0341-21b8-4af5-a19a-f1c4fa5a7f4c,0


In [27]:
merged_df.drop(columns=['PATIENT', 'ID'], inplace=True)
merged_df.head()

,RACE,ETHNICITY,SYSTOLIC_BP,DIASTOLIC_BP,AGE,MEDICATIONS_CODE,ALLERGIES_CODE,IMMUNIZATIONS_CODE,ENCOUNTERS_CODE,TARGET
0,12,36,118.0,86.0,36.0,141918,91934008,140,185349003,0
1,12,36,118.0,86.0,36.0,141918,91934008,140,185349003,0
2,12,36,118.0,86.0,36.0,141918,91934008,140,185349003,0
3,12,36,118.0,86.0,36.0,141918,91934008,140,185349003,0
4,12,36,118.0,86.0,36.0,141918,91934008,140,308335008,0


In [28]:
merged_df.isna().sum()

RACE                  0
ETHNICITY             0
SYSTOLIC_BP           0
DIASTOLIC_BP          0
AGE                   0
MEDICATIONS_CODE      0
ALLERGIES_CODE        0
IMMUNIZATIONS_CODE    0
ENCOUNTERS_CODE       0
TARGET                0
dtype: int64

In [29]:
# Calculate the indices for the 85 splits
# Used 85 so each would have equal parts
split_size = len(merged_df) // 85
splits = [i * split_size for i in range(1, 86)]

# Split the DataFrame into five equal parts based on row index
merged_part1 = merged_df.iloc[splits[9]:splits[11]]
merged_part2 = merged_df.iloc[splits[40]:splits[42]]
merged_part3 = merged_df.iloc[splits[44]:splits[46]]
merged_part4 = merged_df.iloc[splits[52]:splits[54]]
merged_part5 = merged_df.iloc[splits[56]:splits[58]]
merged_part6 = merged_df.iloc[splits[4]:splits[5]]
merged_part7 = merged_df.iloc[splits[5]:splits[6]]
merged_part8 = merged_df.iloc[splits[22]:splits[23]]
merged_part9 = merged_df.iloc[splits[4]:splits[8]]
merged_part10 = merged_df.iloc[splits[18]:splits[19]]
merged_part11 = merged_df.iloc[splits[49]:splits[51]]
merged_part12 = merged_df.iloc[splits[10]:splits[11]]
merged_part13 = merged_df.iloc[splits[71]:splits[72]]
merged_part14 = merged_df.iloc[splits[33]:splits[34]]
merged_part15 = merged_df.iloc[splits[13]:splits[14]]
merged_part16 = merged_df.iloc[splits[64]:splits[65]]
merged_part17 = merged_df.iloc[splits[15]:splits[16]]

In [32]:
target_column = 'TARGET'
df_target_counts = []
# Print the value counts for each un-balanced DataFrame
for i, balanced_df in enumerate([merged_part1, merged_part2, merged_part3, merged_part4, merged_part5, merged_part6, merged_part7, merged_part8, merged_part9, merged_part10]):
    if len(balanced_df[target_column].value_counts()) > 1:
        df_target_counts.append(balanced_df)
        print(f"Balanced DataFrame {i + 1} Class Distribution:\n{balanced_df[target_column].value_counts()}")

Balanced DataFrame 2 Class Distribution:
TARGET
0    4882950
1        704
Name: count, dtype: int64
Balanced DataFrame 3 Class Distribution:
TARGET
0    4866554
1      17100
Name: count, dtype: int64
Balanced DataFrame 4 Class Distribution:
TARGET
0    4831478
1      52176
Name: count, dtype: int64
Balanced DataFrame 5 Class Distribution:
TARGET
0    4864166
1      19488
Name: count, dtype: int64
Balanced DataFrame 9 Class Distribution:
TARGET
0    9766348
1        960
Name: count, dtype: int64


In [33]:
from imblearn.over_sampling import SMOTE
def apply_smote(df, target_column, random_state=42):
    X = df.drop(columns=[target_column])
    y = df[target_column]
    smote = SMOTE(sampling_strategy='auto', random_state=random_state)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    balanced_df = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name=target_column)], axis=1)
    return balanced_df

In [34]:
count = 0
max = len(df_target_counts)

In [35]:
if count < max: 
    balanced_df1 = apply_smote(df_target_counts[count], target_column)
    count = count + 1
    print("Balanced DataFrame 1 Class Distribution:\n", balanced_df1[target_column].value_counts())

Balanced DataFrame 1 Class Distribution:
 TARGET
0    4882950
1    4882950
Name: count, dtype: int64


In [36]:
if count < max:
    balanced_df2 = apply_smote(df_target_counts[count], target_column)
    count = count + 1
    print("Balanced DataFrame 2 Class Distribution:\n", balanced_df2[target_column].value_counts())

Balanced DataFrame 2 Class Distribution:
 TARGET
0    4866554
1    4866554
Name: count, dtype: int64


In [37]:
if count < max:
    balanced_df3 = apply_smote(df_target_counts[count], target_column)
    count = count + 1
    print("Balanced DataFrame 3 Class Distribution:\n", balanced_df3[target_column].value_counts())

Balanced DataFrame 3 Class Distribution:
 TARGET
0    4831478
1    4831478
Name: count, dtype: int64


In [38]:
if count < max:
    balanced_df4 = apply_smote(df_target_counts[count], target_column)
    count = count + 1
    print("Balanced DataFrame 4 Class Distribution:\n", balanced_df4[target_column].value_counts())

Balanced DataFrame 4 Class Distribution:
 TARGET
0    4864166
1    4864166
Name: count, dtype: int64


In [39]:
if count < max: 
    balanced_df5 = apply_smote(df_target_counts[count], target_column)
    count = count + 1
    print("Balanced DataFrame 1 Class Distribution:\n", balanced_df1[target_column].value_counts())

Balanced DataFrame 1 Class Distribution:
 TARGET
0    4882950
1    4882950
Name: count, dtype: int64


In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

base_model = LogisticRegression()

# Initialize the BaggingClassifier
bagging_model = BaggingClassifier(base_model, n_estimators=6, random_state=42)


In [41]:
base_models = []
test_sets = []

In [42]:
X1 = balanced_df1.drop('TARGET', axis=1)
y1 = balanced_df1['TARGET']

In [43]:
X2 = balanced_df2.drop('TARGET', axis=1)
y2 = balanced_df2['TARGET']

In [44]:
X3 = balanced_df3.drop('TARGET', axis=1)
y3 = balanced_df3['TARGET']

In [45]:
X4 = balanced_df4.drop('TARGET', axis=1)
y4 = balanced_df4['TARGET']

In [46]:
X5 = balanced_df5.drop('TARGET', axis=1)
y5 = balanced_df5['TARGET']

In [47]:
min_len = min(len(X1), len(X2), len(X3), len(X4), len(X5))

In [48]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1[:min_len], y1[:min_len], test_size=0.2, random_state=42)
test_sets.append((X_test1, y_test1))

In [49]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2[:min_len], y2[:min_len], test_size=0.2, random_state=42)
test_sets.append((X_test2, y_test2))


In [50]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3[:min_len], y3[:min_len], test_size=0.2, random_state=42)
test_sets.append((X_test3, y_test3))


In [51]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4[:min_len], y4[:min_len], test_size=0.2, random_state=42)
test_sets.append((X_test4, y_test4))


In [52]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(X5[:min_len], y5[:min_len], test_size=0.2, random_state=42)
test_sets.append((X_test5, y_test5))

In [53]:
# Create a new instance of the base model for each iteration
base_model1 = LogisticRegression()

In [54]:
# Create a new instance of the base model for each iteration
base_model2 = LogisticRegression()

In [55]:
# Create a new instance of the base model for each iteration
base_model3 = LogisticRegression()

In [56]:
# Create a new instance of the base model for each iteration
base_model4 = LogisticRegression()

In [57]:
base_model5 = LogisticRegression()

In [58]:
# Train the base model on the subset
base_model1.fit(X_train1, y_train1)

LogisticRegression()

In [59]:
# Train the base model on the subset
base_model2.fit(X_train2, y_train2)

LogisticRegression()

In [60]:
# Train the base model on the subset
base_model3.fit(X_train3, y_train3)

LogisticRegression()

In [61]:
# Train the base model on the subset
base_model4.fit(X_train4, y_train4)

LogisticRegression()

In [62]:
base_model5.fit(X_train4, y_train5)

LogisticRegression()

In [63]:
# Add the trained model to the list
base_models.append(base_model1)

In [64]:
# Add the trained model to the list
base_models.append(base_model2)

In [65]:
# Add the trained model to the list
base_models.append(base_model3)

In [66]:
# Add the trained model to the list
base_models.append(base_model4)

In [67]:
base_models.append(base_model5)

In [68]:
y_pred1 = base_model1.predict(X_test1)

In [69]:
y_pred2 = base_model2.predict(X_test2)

In [70]:
y_pred3 = base_model3.predict(X_test3)

In [71]:
y_pred4 = base_model4.predict(X_test4)

In [72]:
y_pred5 = base_model5.predict(X_test5)

In [73]:
print(f"Model 1 Report: {classification_report(y_test1, y_pred1)}")
print(f"Accuracy on Test Set 1: {accuracy_score(y_test1, y_pred1)}")

Model 1 Report:               precision    recall  f1-score   support

           0       0.91      0.71      0.80    976748
           1       0.76      0.93      0.83    955844

    accuracy                           0.82   1932592
   macro avg       0.83      0.82      0.81   1932592
weighted avg       0.83      0.82      0.81   1932592

Accuracy on Test Set 1: 0.8156884639903301


In [74]:
print(f"Model 2 Report: {classification_report(y_test2, y_pred2)}")
print(f"Accuracy on Test Set 2: {accuracy_score(y_test2, y_pred2)}")

Model 2 Report:               precision    recall  f1-score   support

           0       0.66      0.81      0.73    973495
           1       0.75      0.57      0.65    959097

    accuracy                           0.69   1932592
   macro avg       0.70      0.69      0.69   1932592
weighted avg       0.70      0.69      0.69   1932592

Accuracy on Test Set 2: 0.6920622666346544


In [75]:
print(f"Model 3 Report: {classification_report(y_test3, y_pred3)}")
print(f"Accuracy on Test Set 3: {accuracy_score(y_test3, y_pred3)}")

Model 3 Report:               precision    recall  f1-score   support

           0       0.60      0.70      0.64    966570
           1       0.63      0.53      0.58    966022

    accuracy                           0.61   1932592
   macro avg       0.61      0.61      0.61   1932592
weighted avg       0.61      0.61      0.61   1932592

Accuracy on Test Set 3: 0.6117494018396019


In [76]:
print(f"Model 4 Report: {classification_report(y_test4, y_pred4)}")
print(f"Accuracy on Test Set 4: {accuracy_score(y_test4, y_pred4)}")

Model 4 Report:               precision    recall  f1-score   support

           0       0.71      0.73      0.72    973061
           1       0.72      0.70      0.71    959531

    accuracy                           0.72   1932592
   macro avg       0.72      0.72      0.72   1932592
weighted avg       0.72      0.72      0.72   1932592

Accuracy on Test Set 4: 0.7157729101641733


In [77]:
print(f"Model 5 Report: {classification_report(y_test5, y_pred5)}")
print(f"Accuracy on Test Set 5: {accuracy_score(y_test5, y_pred5)}")

/Users/shawngoforth/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shawngoforth/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shawngoforth/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model 5 Report:               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1932398
           1       0.00      0.00      0.00       194

    accuracy                           1.00   1932592
   macro avg       0.50      0.50      0.50   1932592
weighted avg       1.00      1.00      1.00   1932592

Accuracy on Test Set 5: 0.999899616680603


In [79]:
# Combine predictions using simple averaging for classification
ensemble_predictions = (
    y_pred1 + y_pred2 + y_pred3 + y_pred4 + y_pred5
) / 5

# Round predictions for classification
y_pred_combined = ensemble_predictions.round().astype(int)

# Evaluate the ensemble model using labels from the first test set
accuracy_combined = accuracy_score(test_sets[0][1], y_pred_combined)
print(f"Combined Model Report: {classification_report(test_sets[0][1], y_pred_combined)}")
print(f"Accuracy on Combined Test Sets: {accuracy_combined}")


Combined Model Report:               precision    recall  f1-score   support

           0       0.71      0.94      0.81    976748
           1       0.91      0.62      0.74    955844

    accuracy                           0.78   1932592
   macro avg       0.81      0.78      0.77   1932592
weighted avg       0.81      0.78      0.77   1932592

Accuracy on Combined Test Sets: 0.780578621871559
